In [ ]:
from datasets import load_dataset

dataset = load_dataset("umarbutler/open-australian-legal-qa")
documents_dataset = load_dataset("umarbutler/open-australian-legal-corpus")

In [ ]:
dataset

In [ ]:
from tqdm import tqdm

corpus_dict = {item['version_id']: item['text'] for item in tqdm(documents_dataset['corpus'])}

In [ ]:
import json

print(json.dumps(dataset['train'][1], indent=2))

In [36]:
from datasets import DatasetDict

dataset = DatasetDict({
    'test': dataset['train']
})

In [ ]:
# import spacy
import os
import re

num_proc=os.cpu_count()

# from more_itertools import windowed
# nlp = spacy.load("en_core_web_sm")

# def chunk_document(document):
#     doc = nlp(document)
#     sentences = [sent.text for sent in doc.sents]
#     return sentences

def process_record(record):
    gold_text = record['answer']
    source = record['source']
    previous_text = record['question']
    document_id = source['version_id']
    source_citation = source['citation']
    if document_id not in corpus_dict:
        return {
            'previous_text': previous_text,
            'gold_text': gold_text,
            'citations': [[source_citation, ""]],
            'valid': False, 
        }
    document = re.sub(r'\n+', '\n', corpus_dict[document_id])
    citations = [[source_citation, document]]
    # sentences = chunk_document(source_document)
    # oracle_documents_passages = []
    # for sentence in windowed(sentences, 5, fillvalue="", step=2):
    #     section = " ".join([s for s in sentence if (s is not None and len(s.strip()) > 0)])
    #     oracle_documents_passages.append([document_id, section])
    return {
        'previous_text': previous_text,
        'gold_text': gold_text,
        'citations': citations,
        'valid': True,
        # 'oracle_documents_passages': oracle_documents_passages,
    }

dataset = dataset.map(process_record, num_proc=num_proc)

In [ ]:
dataset = dataset.map(lambda _, idx: {'docid': str(idx + 1)}, with_indices=True)

In [ ]:
new_dataset = DatasetDict({
    'train': dataset['test'].filter(lambda x: not x['valid']),
    'test': dataset['test'].filter(lambda x: x['valid'])
})
new_dataset

In [ ]:
new_dataset = new_dataset.select_columns(['docid', 'previous_text', 'gold_text', 'citations'])
new_dataset

In [ ]:
new_dataset.push_to_hub("ylkhayat/OAL_QA-generation-workshop", data_dir="data")